# Creating the containers for .NET notebooks for exploring dbatools

To run these notebooks, you will first have to have set up Jupyter Notebooks following the information here https://sqldbawithabeard.com/2020/02/07/new-net-notebooks-are-here-powershell-7-notebooks-are-here/

![dbatools](.\images\dbatools.png )

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

# Setting up the containers for the rest of the dbachecks notebooks

To be able to follow along with the rest of the notebooks, you will need to set up two containers. 

This notebook will enable you to do that.

You will need to have Docker installed. The image is based on the SQL Server 2019 image so you will need to have docker set to use Linux Containers. Right click on the Docker icon in the notification area and if it says "Switch to Linux Containers" click to switch and wait for Docker to restart.  

![Switch To Linux Containers](.\images\switchtolinuxcontainers.png )

You will be able to run all of the code in the notebooks by creating the folder, credential and containers in this notebook and then you can click on the play button in each code block to run the code. Note - There are a few code blocks with the results already included which should not be run. They are to show you the results of a command that cannot be run against containers (setting up configuration for domain accounts for example)

**You can also navigate through the Notebook using the arrow keys and press SHIFT + ENTER to run a cell**

## Create the folders and the credential

The code below will create a directory called dbatools-demo in your Documents folder and save a credential file for logging into the containers. You can alter the directory created by altering the `$FolderPath`  

The code has been collapsed but you can view it using the view menu in the lab or by clicking the 3 ellipses

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'

########################################################
Write-Output "Creating Directory $FolderPath"
if(Test-Path $FolderPath){
    Write-Output "Path $FolderPath exists already"
}else {
 $null =    New-Item $FolderPath -ItemType Directory
}

Write-Output "Creating Directory $FolderPath\SQL1"
if(Test-Path "$FolderPath\SQL1"){
    Write-Output "Directory SQL1 exists already"
    Get-ChildItem "$FolderPath\SQL1" -Recurse| Remove-Item -Recurse -Force
}else {
$null = New-Item "$FolderPath\SQL1"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL1\dummyfile.txt"
if(Test-Path "$FolderPath\SQL1\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL1\dummyfile.txt" -ItemType file
}

Write-Output "Creating Directory $FolderPath\SQL2"
if(Test-Path "$FolderPath\SQL2"){
    Write-Output "Directory SQL2 exists already"
    Get-ChildItem "$FolderPath\SQL2" -Recurse| Remove-Item -Recurse -Force
}else {
 $null =    New-Item "$FolderPath\SQL2"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL2\dummyfile.txt"
if(Test-Path "$FolderPath\SQL2\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL2\dummyfile.txt" -ItemType file
}

Write-Output "Creating a credential file for the containers - Please don't do this in production"

$sqladminPassword = ConvertTo-SecureString 'dbatools.IO' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('sqladmin', $sqladminpassword)
$Cred | Export-Clixml -Path $FolderPath\sqladmin.cred
Write-Output "Credential file created"

Write-Output "Setting the docker-compose files values"
try {
    $dockercompose = (Get-Content .\dockercompose.yml -ErrorAction Stop) -replace '__ReplaceME__' , $FolderPath
    # $dockercompose
    $dockercompose | Set-Content .\docker-compose.yml
}
catch {
    $NotebookDir = Read-Host "Please enter path to the notebook. Azure Data Studio doesnt know where it is"
    $dockercompose = (Get-Content $NotebookDir\\dockercompose.yml -ErrorAction Stop) -replace '__ReplaceME__' , $FolderPath
    # $dockercompose
    $dockercompose | Set-Content $NotebookDir\\docker-compose.yml
    Set-Location $NotebookDir
}

Write-Output "Finished"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo
Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo\SQL1
Creating File C:\Users\mrrob\Documents\dbatoolsdemo\SQL1\dummyfile.txt
Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo\SQL2
Creating File C:\Users\mrrob\Documents\dbatoolsdemo\SQL2\dummyfile.txt
Creating a credential file for the containers - Please don't do this in production
Credential file created
Setting the docker-compose files values
Finished


## Containers

We are going to create two SQL 2019 containers using the sqldbawithabeard\dbachecks [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbachecksdemo). 

The first time it is going to pull the image sqldbawithabeard/dbachecksdemo from the Docker Hub. If you wish to do this first, you can run the cell below



In [1]:
docker pull sqldbawithabeard/dbachecksdemo

Using default tag: latest
latest: Pulling from sqldbawithabeard/dbachecksdemo
b234f539f7a1: Pulling fs layer
55172d420b43: Pulling fs layer
5ba5bbeb6b91: Pulling fs layer
43ae2841ad7a: Pulling fs layer
f6c9c6de4190: Pulling fs layer
fd449e8d7345: Pulling fs layer
51d0933375e5: Pulling fs layer
64f21ba81504: Pulling fs layer
55b6919c0cc6: Pulling fs layer
76386ba2eb1b: Pulling fs layer
43ae2841ad7a: Waiting
64f21ba81504: Waiting
55b6919c0cc6: Waiting
f6c9c6de4190: Waiting
76386ba2eb1b: Waiting
fd449e8d7345: Waiting
51d0933375e5: Waiting
5ba5bbeb6b91: Verifying Checksum
5ba5bbeb6b91: Download complete
55172d420b43: Verifying Checksum
55172d420b43: Download complete
f6c9c6de4190: Verifying Checksum
f6c9c6de4190: Download complete
43ae2841ad7a: Verifying Checksum
43ae2841ad7a: Download complete
b234f539f7a1: Download complete
fd449e8d7345: Verifying Checksum
fd449e8d7345: Download complete
b234f539f7a1: Pull complete
55172d420b43: Pull complete
5ba5bbeb6b91: Pull complete
43ae2841ad7a: Pul

Now we can start the containers with the code below

In [2]:
docker-compose up -d

dbatools_SQL2019-1_1 is up-to-date
dbatools_SQL2019_1 is up-to-date


All being well, you wil have something that looks like


![DockerCompose](.\images\containers.png )

If you get an error you might need to add the user you have shared your drives with Docker modify permissions to the \Documents\ directory in your user profile   
More details https://docs.docker.com/docker-for-windows/ or troubleshoot in the normal way


Now we can start exploring with dbatools :-)

If you have not installed dbatools, it can be got from the PowerShell Gallery using `Install-Module dbatools` the code below will check for the module and either install it in your user profile or update it and Import it

In [2]:
if(Get-Module dbatools -ListAvailable){
Write-Output "Updating dbatools"
Update-Module dbatools
}else {
Write-Output "Installing dbatools in your user profile"
Install-Module dbatools -Scope CurrentUser
}
Import-Module dbatools

Installing dbatools in your user profile


                                                                                                    
 Installing package 'dbatools'                                                                      
                                                                                                    
    [                                                                                        ]      
                                                                                                    

                                                                                                    
 Installing package 'dbatools'                                                                      
    Downloaded 0.00 MB out of 24.91 MB.                                                             
    [                                                                                        ]      
                                                                                                    

                                                                                                    
 Installing package 'dbatools'                                                                      
    Downloaded 15.00 MB out of 24.91 MB.                                                            
    [oooooooooooooooooooooooooooooooooooo                                                    ]      
                                                                                                    

                                                                                                    
 Installing package 'dbatools'                                                                      
    Unzipping                                                                                       
    [ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo                           ]      
                                                                                                    

                                                                                                    
 Installing package 'dbatools'                                                                      
    Copying unzipped package to 'C:\Users\mrrob\AppData\Local\Temp\1865382987\dbatools.1.0.113'     
    [oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo        ]      
                                                                                                    

                                                                                                    
 Installing package 'dbatools'                                                                      
    Process Package Manifest                                                                        
    [ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo     ]      
                                                                                                    

Import-Module: 
Line |
   8 |  Import-Module dbatools
     |  ~~~~~~~~~~~~~~~~~~~~~~
     | Could not find a part of the path 'C:\Users\mrrob\OneDrive\Documents\PowerShell\Modules\dbatools\1.0.108\xml\dbatools.Format.ps1xml'.


## Check connection
Now that is done, we can make a connection to our instances and see if we can connect to them.

We are going to use `Connect-DBaInstance` to do this and we will use the containers that we have created and because we need to use SQL Authentication, we will use a credential that we have saved to disk using `Export-CliXML` in one of the celss above.

It is **Important** to state that this is not a production secure solution and should not be used as a means of accessing any secure system.

In [3]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

 Creating connection to the containers
We have a connection to the containers


If there are warnings above - Check the containers are running - Look at the status column

In [4]:
docker ps -a

CONTAINER ID        IMAGE                            COMMAND                  CREATED             STATUS              PORTS                                             NAMES
b7ee5654561f        sqldbawithabeard/dbachecksdemo   "/opt/mssql/bin/perm…"   9 minutes ago       Up 9 minutes        0.0.0.0:5022->5022/tcp, 0.0.0.0:15592->1433/tcp   dbatools_SQL2019_1
31f2db5347e2        sqldbawithabeard/dbachecksdemo   "/opt/mssql/bin/perm…"   9 minutes ago       Up 9 minutes        0.0.0.0:5023->5023/tcp, 0.0.0.0:15593->1433/tcp   dbatools_SQL2019-1_1


If there are warnings above and the containers are running - check the logs

In [5]:
docker logs dbatools_SQL2019_1

SQL Server 2019 will run as non-root by default.
This container is running as user mssql.
Your master database file is owned by mssql.
To learn more visit https://go.microsoft.com/fwlink/?linkid=2099216.
2020-06-17 18:16:23.40 Server      Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) 

	Sep 24 2019 13:48:23 

	Copyright (C) 2019 Microsoft Corporation

	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>

2020-06-17 18:16:23.41 Server      UTC adjustment: 0:00

2020-06-17 18:16:23.41 Server      (c) Microsoft Corporation.

2020-06-17 18:16:23.41 Server      All rights reserved.

2020-06-17 18:16:23.41 Server      Server process ID is 32.

2020-06-17 18:16:23.41 Server      Logging SQL Server messages in file '/var/opt/mssql/log/errorlog'.

2020-06-17 18:16:23.41 Server      Registry startup parameters: 

	 -d /var/opt/mssql/data/master.mdf

	 -l /var/opt/mssql/data/mastlog.ldf

	 -e /var/opt/mssql/log/errorlog

2020-06-17 18:16:23.41 Server      SQL Server detected 1 so

If there are warnings above and the containers are running - check the logs

In [ ]:
docker logs dbatools_SQL2019_1

Now you can use these containers to run the rest of the notebooks - Don't forget to run the Clean Up Containers Notebook at the end to clean up